In [ ]:
# | default_exp production.markings

# Production.Markings

In [ ]:
# | export

from p4tools.production import io
from p4tools.markings import Blotch,Fan

import logging
import matplotlib.pyplot as plt
import itertools
import pandas as pd


LOGGER = logging.getLogger(__name__)

In [ ]:
# | export 
IMG_X_SIZE = 840
IMG_Y_SIZE = 648

IMG_SHAPE = (IMG_Y_SIZE, IMG_X_SIZE)

GOLD_MEMBERS = ['michaelaye', 'mschwamb', 'Portyankina']  # 'CJ-DPI']
GOLD_PLOT_COLORS = list('bmg')

In [ ]:
# | export

def set_subframe_size(ax):
    """Set plot view limit on Planet 4 subframe size."""
    ax.set_xlim(0, IMG_X_SIZE)
    ax.set_ylim(IMG_Y_SIZE, 0)

def calc_fig_size(width):
    """Calc figure height in ratio of subframes."""
    ratio = IMG_X_SIZE / IMG_Y_SIZE
    return (width, width / ratio)

In [ ]:
# | export

class TileID:

    """Manage Planet 4 Image ids, getting data, plot stuff etc.

    At init this class will get the data for the given `imgid` using either the latest
    found database file or the optionally provided one.
    Parameters
    ----------
    imgid : str
        Planet4 image_id
    dbname : str, optional
        Filepath to database name. The marking data for `imgid` will be extracted.
        Default: Latest one.
    data : pd.DataFrame, optional
        If the data was already extracted before init, it can be provided here.
    """

    def __init__(self, imgid, scope='planet4', dbname=None, data=None, image_name=None):
        self.imgid = io.check_and_pad_id(imgid)
        self._data = data
        self.scope = scope
        self.dbname = dbname
        self._image_name = image_name

    @property
    def data(self):
        if self._data is not None:
            return self._data
        try:
            db = io.DBManager(self.dbname)
            self._data = db.get_image_id_markings(self.imgid, self.image_name)
            return self._data
        except NoFilesFoundError:
            print("Cannot find PlanetFour database.")
            return None

    @property
    def image_name(self):
        if self._image_name is None:
            db = io.DBManager(self.dbname)
            self._image_name = db.get_obsid_for_tile_id(self.imgid)
        return self._image_name

    @property
    def tile_coords(self):
        return self.data[['x_tile', 'y_tile']].drop_duplicates().values[0]

    @property
    def blotchmask(self):
        return self.data.marking == 'blotch'

    @property
    def fanmask(self):
        return self.data.marking == 'fan'

    @property
    def n_marked_classifications(self):
        return self.data[self.blotchmask | self.fanmask].classification_id.nunique()

    @property
    def subframe(self):
        "np.array : Get tile url and return image tile using io funciton."
        url = self.data.iloc[0].image_url
        return io.get_subframe(url)

    def filter_data(self, kind, user_name=None, without_users=None):
        """Filter and return data for kind, user, or without_users.

        Parameters
        ----------
        kind : {'fan', 'blotch'}
            Marking
        user_name : str
            Filter data further for `user_name`
        without_users : list(strings)
            Only return data that is not in list of user_names (useful for non-gold data)
        """
        mask = self.data.marking == kind
        if user_name is not None:
            mask = (mask) & (self.data.user_name == user_name)
        if without_users is not None:
            mask = (mask) & (~self.data.user_name.isin(without_users))
        return self.data[mask]

    def get_fans(self, user_name=None, without_users=None):
        """Return data for fan markings."""
        return self.filter_data('fan', user_name, without_users)

    def get_blotches(self, user_name=None, without_users=None):
        """Return data for blotch markings."""
        return self.filter_data('blotch', user_name, without_users)

    def show_subframe(self, ax=None, aspect='auto'):
        if ax is None:
            fig, ax = plt.subplots(figsize=calc_fig_size(8))
        ax.imshow(self.subframe, origin='upper', aspect=aspect)
        ax.set_axis_off()

    def plot_objects(self, objects, n=None, img=True, ax=None,
                     user_color=None, user_colors=None, wind_pointer=False):
        """Plotting either fans or blotches with p4 subframe background."""
        LOGGER.debug("Entering markings.plot_objects")
        LOGGER.debug("Received %i objects to plot.", len(objects))
        if ax is None:
            _, ax = plt.subplots(figsize=calc_fig_size(8))
            LOGGER.debug("Created own axis.")
        if img:
            LOGGER.debug("Plotting background image.")
            self.show_subframe(ax)
        counter = 0
        if user_colors is None:
            colors = itertools.cycle(sns.color_palette('bright', 12))
        else:
            colors = user_colors
        for obj, color in zip(objects, colors):
            if user_color is not None:
                color = user_color
            obj.plot(color=color, ax=ax)
            if wind_pointer is True and isinstance(obj, Fan):
                obj.add_mean_wind_pointer(color=color, ax=ax)
            counter += 1
            if counter == n:
                break
        set_subframe_size(ax)
        ax.set_axis_off()

    def pop_kwargs(self, kwargs):
        with_center = kwargs.pop('with_center', False)
        user_name = kwargs.pop('user_name', None)
        without_users = kwargs.pop('without_users', None)
        lw = kwargs.pop('lw', 1)
        return with_center, user_name, without_users, lw

    def plot_markings(self, kind, data=None, **kwargs):
        """General plotting method for both fans and blotches.

        Also accepts any derived set of objects in the `data` parameter.

        Parameters:
        ----------
        kind : {'blotch','fan'}
            Switch between specialized plotting features
        **kwargs : {dict}
            Any keywords to hand down to other methods here and/or matplotlib
        data : {list, pd.core.frame.DataFrame}, optional
            Any marking objects related to this image_id (the default is None, which will plot the raw data.)

        """
        with_center, user_name, without_users, lw = self.pop_kwargs(kwargs)
        if data is None:
            data = self.filter_data(kind, user_name, without_users)
        markingClass = Blotch if kind == 'blotch' else Fan
        if type(data) == pd.core.frame.DataFrame:
            data = [markingClass(i, self.scope, with_center=with_center, lw=lw)
                    for _, i in data.iterrows()]
        self.plot_objects(data, **kwargs)

    def plot_blotches(self, data=None, **kwargs):
        self.plot_markings('blotch', data=data, **kwargs)

    def plot_fans(self, data=None, **kwargs):
        self.plot_markings('fan', data=data, **kwargs)

    def plot_all(self, savedir=None):
        fig, axes = plt.subplots(2, 2)
        axes = axes.ravel()
        for i in [0, 2]:
            self.show_subframe(ax=axes[i])
        self.plot_fans(ax=axes[1])
        self.plot_blotches(ax=axes[3])
        for ax in axes:
            ax.set_axis_off()
        # fig.subplots_adjust(left=None, top=None, bottom=None, right=None,
                            # wspace=1e-3, hspace=1e-3)
        fig.suptitle(self.imgid)
        if savedir is not None:
            savepath = savedir / f"{self.imgid}.png"
            fig.savefig(savepath, dpi=150)

